In [2]:
import pandas as pd
import ast

# Load the recipes file (accessing from parent directory)
recipes_df = pd.read_csv('../recipes_with_labels.csv')

# Load the ingredients labels file (accessing from parent directory)
labels_df = pd.read_csv('../labels/ingredients.csv')

# Convert NER column from string representation to actual lists
recipes_df['NER'] = recipes_df['NER'].apply(ast.literal_eval)

# Create a dictionary for ingredient synonyms and their categories
ingredient_categories = {}
for _, row in labels_df.iterrows():
    synonyms = row['Ingredient Synonyms'].split('; ')
    for synonym in synonyms:
        ingredient_categories[synonym.lower()] = row['Category']
    # Add the aliased ingredient name as well
    ingredient_categories[row['Aliased Ingredient Name'].lower()] = row['Category']

# Function to categorize ingredients in a recipe based on NER
def categorize_ingredients(ner_list):
    categories = set()
    for ingredient in ner_list:
        # Normalize the ingredient to lowercase for matching
        normalized_ingredient = ingredient.lower()
        for key in ingredient_categories:
            # Skip if the category or key is 'dish' or 'cereal'
            if key == 'dish' or key == 'cereal':
                continue
            if key in normalized_ingredient:
                category = ingredient_categories[key]
                if category.lower() != 'dish' and category.lower() != 'cereal':
                    categories.add(category)
    return list(categories)

# Overwrite categories column with new values derived from NER
recipes_df['categories'] = recipes_df['NER'].apply(categorize_ingredients)

# Save the updated DataFrame to a new CSV file in parent directory
recipes_df.to_csv('../recipes_with_labels.csv', index=False)

# Display a sample of the updated DataFrame
print(recipes_df[['title', 'NER', 'categories']].head())

                   title                                                NER  \
0    no-bake nut cookies  [bite size shredded rice biscuits, vanilla, br...   
1  jewell ball's chicken  [cream of mushroom soup, beef, sour cream, chi...   
2            creamy corn  [frozen corn, pepper, cream cheese, garlic pow...   
3          chicken funny  [chicken gravy, cream of mushroom soup, chicke...   
4   reeses cups(candy)    [graham cracker crumbs, powdered sugar, peanut...   

                                          categories  
0                   [Additive, Dairy, Bakery, Fruit]  
1                              [Dairy, Meat, Fungus]  
2              [Additive, Dairy, Herb, Spice, Maize]  
3                              [Dairy, Meat, Fungus]  
4  [Additive, Plant, Nuts & Seed, Dairy, Beverage...  
